# Exploring Twitter Data with Vector Databases and RAG Systems

This tutorial introduces the creation of a vector database and the use of a retrieval-augmented generation (RAG) system to explore Twitter data interactively. 

Please check [LBSocial](www.lbsocial.net)  on how to collect Twitter data. 

## Set up a Database and API Keys

Create a [MongoDB](www.mongodb.com) cluster and store the connection string in a safe place, such as AWS Secrets Manager. 
- key name: `connection_string`
- key value: <`the connection string`>, you need to type the password
- secret name: `mongodb`


You also need to purchase and your [oepnai](https://openai.com/) api key in AWS Secrets Manager:
- key name: `api_key`
- key value: <`your openai api key`>
- secret name: `openai`

## Install Python Libraries

- pymongo: manage the MongoDB database
- openai: create embeddings and resonpses 

In [1]:
pip install pymongo openai -q

Note: you may need to restart the kernel to use updated packages.


## Secrets Manager Function

In [2]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Import Python Libraries and Credentials

In [3]:
import pymongo
from pymongo import MongoClient
import json
import re
import os
from openai import OpenAI
openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)


mongodb_connect = get_secret('mongodb')['connection_string']


## Connect to the MongoDB cluster

In [4]:
mongo_client = MongoClient(mongodb_connect)
db = mongo_client.demo # use or create a database named demo
tweet_collection = db.tweet_collection #use or create a collection named tweet_collection

## Utility Funcitons

- the `clean_tweet` function removes URLs in tweets
- the `get_embedding` function use openai to create tweet embeddings
- the `vector_search` function return relevent tweets based on a query

In [5]:
def clean_tweet(text):
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    return re.sub(url_pattern, '', text)

In [6]:
embedding_model= 'text-embedding-3-small'

def get_embedding(text):

    try:
        embedding = client.embeddings.create(input=text, model=embedding_model).data[0].embedding
        return embedding
    except Exception as e:
        print(f"Error in get_embedding: {e}")
        return None

In [7]:
def vector_search(query):

    query_embedding = get_embedding(query)
    if query_embedding is None:
        return "Invalid query or embedding generation failed."
    # Define the vector search pipeline
    pipeline = [
        {
            "$vectorSearch": {
                "index": "tweet_vector",
                "queryVector": query_embedding,
                "path": "tweet.embedding",
                "numCandidates": 1000,  # Number of candidate matches to consider
                "limit": 10  # Return top 10 matches
            }
        },
        {
            "$project": {
                "_id": 0,  # Exclude the _id field
                "tweet.text": 1 # return tweet text

            }
        }
    ]

    results = tweet_collection.aggregate(pipeline)
    return list(results)

## Tweets Embedding 

For more about text embeddings please read [Introducing text and code embeddings](https://openai.com/index/introducing-text-and-code-embeddings/)

In [8]:
from tqdm.auto import tqdm
tweets = tweet_collection.find()

for tweet in tqdm(list(tweets)):
    try:
        tweet_embedding = get_embedding(clean_tweet(tweet['tweet']['text']))
    #     print(tweet_embedding)

        tweet_collection.update_one(
            {'tweet.id':tweet['tweet']['id']},
            {"$set":{'tweet.embedding':tweet_embedding}}
        )
    except:
        print(f"""error in embedding tweet {tweet['tweet']['id']}""")
        pass


  0%|          | 0/99 [00:00<?, ?it/s]

## Create a Vector Index

For more about the MognoDB Vector database, please read [What are Vector Databases?](https://www.mongodb.com/resources/basics/databases/vector-databases)
This code creates a vector index following the [MongoDB official document](https://www.mongodb.com/docs/atlas/atlas-vector-search/vector-search-type/#std-label-avs-types-vector-search).

In [9]:
# Create your index model, then create the search index

from pymongo.operations import SearchIndexModel
import time

search_index_model = SearchIndexModel(
  definition={
  "fields": [
    {
      "type": "vector",
      "path": "tweet.embedding",
      "numDimensions": 1536,
      "similarity": "cosine"
    }
  ]
},
  name="tweet_vector",
  type="vectorSearch"

)
result = tweet_collection.create_search_index(model=search_index_model)
print("New search index named " + result + " is building.")
# Wait for initial sync to complete
print("Polling to check if the index is ready. This may take up to a minute.")
predicate=None

if predicate is None:
  predicate = lambda index: index.get("queryable") is True

while True:
  indices = list(tweet_collection.list_search_indexes(result))
  if len(indices) and predicate(indices[0]):

    break
  time.sleep(5)

print(result + " is ready for querying.")

New search index named tweet_vector is building.
Polling to check if the index is ready. This may take up to a minute.
tweet_vector is ready for querying.


In [16]:
user_query = 'ai news'

for tweet in vector_search(user_query):
    print(tweet['tweet']['text'])

RT @Mastardcesh: "The AU will act on the investigation of the election protests in Tanzania."

— Ali Mohamoud https://t.co/cqtplmkMFc
RT @Mastardcesh: "The AU will act on the investigation of the election protests in Tanzania."

— Ali Mohamoud https://t.co/cqtplmkMFc
RT @nbstv: .@ckaheru: This machine is a newer version of the previous ones we have seen. I think we should test it and see if it can make t…
RT @mitchellvii: IS JAVIER MILEI - AMERICA'S TOP BUDDY - CRUSHING THE CHEATERS WITH HIS GENIUS SINGLE BALLOT FIX FOR A FAIR FIGHT IN ARGENT…
RT @atensnut: 🚨 This could decide the governor election in New Jersey!!
RT @GoldenAgeTimes2: 🚨NJ ALERT: New &amp; Infrequent voters are SURGING to the polls, confident they will match Dems' Voting numbers by Electio…
RT @IvankaNews_: A Significant Shift in Momentum: Can the Former Governor Make a Comeback in the New York City Mayoral Election Next Week?
RT @OfficialAPCNg: ANAMBRA GUBER NATIONAL ELECTION MANAGEMENT COUNCIL

INAUGURATION
Date: Frid

## Retrieval-Augmented Generation (RAG) 

For more about RAG, please read [Retrieval-Augmented Generation (RAG) with Atlas Vector Search](https://www.mongodb.com/docs/atlas/atlas-vector-search/rag/#std-label-avs-rag).

In [17]:
from openai import OpenAI

delimiter = '###'
chat_model = 'gpt-4o'
temperature = 0

chat_history = [{"role": "system", "content": """ you are a chatbot, answer user questions based on the returned tweets."""}]

def chatbot(prompt):

    chat_history.append({"role": "user", "content": prompt})
    
    tweets = vector_search(prompt)
    chat_history.append({"role": "system", "content": f" here the returned tweets delimited by {delimiter}{tweets}{delimiter}"})

    response = client.chat.completions.create(
        model=chat_model,  # Use the model you prefer
        messages=chat_history
    )

    reply = response.choices[0].message.content

    chat_history.append({"role": "assistant", "content": reply})
    
    return reply

In [18]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        print("Chatbot: Goodbye!")
        break
    reply = chatbot(user_input)
    print(f"Chatbot: {reply}")

You:  study ai


Chatbot: It seems the tweets provided are focused on election topics, such as issues or events surrounding elections in different regions. If you're interested in studying AI, you might want to consider exploring:

1. **Natural Language Processing (NLP):** Understanding how AI systems analyze and understand human language can be crucial, especially if you're aiming to analyze tweets and other social media data.

2. **Machine Learning (ML) Models:** You can develop models to predict election outcomes, detect fraudulent activities, or analyze voter sentiment using various supervised and unsupervised learning techniques.

3. **Sentiment Analysis:** This involves using AI to determine the sentiment or emotional tone behind texts—something that can be particularly useful in understanding public opinion from tweets.

4. **Data Collection and Preparation:** Learning how to collect, clean, and preprocess data from social media is a fundamental skill in applying AI to real-world problems.

5. *

You:  study election topics


Chatbot: If you're interested in studying election topics, here are some themes and areas of focus that emerge from the tweets:

1. **Candidate Profiles and Elections:** Understanding the profiles and personalities of candidates in elections can provide insights into voter decision-making. Research and analysis into candidate backgrounds, policies, and performance can inform predictions and discussions about election outcomes.

2. **Voter Turnout and Engagement:** Tweets mention the importance of every vote and efforts to encourage voter participation, such as taking friends to the polls. Studying voter turnout patterns and strategies for increasing participation can be critical in analyzing election dynamics.

3. **Fraud and Election Integrity:** Concerns about fraudulent votes and election integrity appear in the tweets. This area involves understanding the mechanisms for ensuring fair elections and detecting or preventing electoral fraud.

4. **Polling and Predictive Analysis:** The

You:  what about protests and political movements


Chatbot: The tweets highlight various aspects of protests and political movements related to elections:

1. **Protests Against Elections:** In some regions, like Tanzania, there are protests not just against election results but against the electoral process itself. This indicates dissatisfaction with how elections are conducted, possibly due to perceived unfairness or lack of transparency.

2. **Youth Participation and Protests:** Younger generations, such as Gen Z in Tanzania, are choosing to protest rather than participate in elections they view as biased or undemocratic. This highlights the role of youth as a significant force in political activism and change.

3. **Response to Election Results:** In Cameroon, protests erupted following the announcement of election results where a long-standing leader, Paul Biya, claimed victory. Such protests underscore the public's objection to perceived electoral fraud or manipulation.

4. **International and Regional Reactions:** The African Un

You:  how do politics in the U.S. influence society 


Chatbot: The tweets provide several insights into how politics in the U.S. influence society:

1. **Shift in Ideological Trends:** There is discussion about young Americans potentially shifting political ideologies, suggesting that U.S. politics can influence cultural and generational change. This shift might affect future policies and the political landscape.

2. **Economic Policies and Social Welfare:** The impact of political decisions on social welfare programs is highlighted, such as the reliance on food stamps and how government spending choices influence societal well-being. Political priorities can directly affect poverty levels and access to essential services.

3. **Empowerment and Civic Engagement:** Political figures like Alexandria Ocasio-Cortez emphasize empowering groups, such as young voters, to engage in civic processes on issues they care about, fostering a sense of agency in societal development.

4. **Electoral Participation:** The importance of voter turnout in ele

You:  quit


Chatbot: Goodbye!


## Reference

- *“Introducing Text and Code Embeddings.”* n.d. OpenAI. Accessed October 31, 2024. https://openai.com/index/introducing-text-and-code-embeddings/.
- *“What Are Vector Databases?”* n.d. MongoDB. Accessed October 31, 2024. https://www.mongodb.com/resources/basics/databases/vector-databases.
